In [1]:
import torch
from torch import nn, optim
import math
import os
from torchinfo import summary

In [ ]:
class DenseNet(nn.Module):
    def __init__(self, n_classes=1000):
        pass
    def forward(self, x):
        return x

In [ ]:
def densenet121(n_classes=1000):
    return DenseNet(n_classes=n_classes)
def densenet161(n_classes=1000):
    return DenseNet(n_classes=n_classes)
def densenet169(n_classes=1000):
    return DenseNet(n_classes=n_classes)
def densenet201(n_classes=1000):
    return DenseNet(n_classes=n_classes)

In [ ]:
d121 = densenet121()
d161 = densenet161()
d169 = densenet169()
d201 = densenet201()

In [ ]:
models = [d121, d161, d169, d210]

In [ ]:
def fmat(n):
    return "{:.2f}M".format(n / 1e6)

In [ ]:
def params(model, f=True):
    s = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return fmat(s) if f else s

In [ ]:
import os
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [ ]:
for m in models:
    print(params(m))

In [ ]:
for m in models:
    print_size_of_model(m)